# Big Idea 3 (Data Structures including List, Dictionaries, 2D arrays and Iteration)

## Leaderboard Database (Kaiden and Navan)
- This is the Leaderboard Class and this part is declaring the keys for the data within the class 
    - ![](https://user-images.githubusercontent.com/69410126/232369087-905a0d82-4bc9-44c8-8f34-1a40543b5783.png)
- These are the setters and getters for the values: username, pointsEasy, pointsMedium, and pointsHard
    - ![](https://user-images.githubusercontent.com/69410126/232369492-e04cf294-5805-468c-8414-402db8b85739.png)
    - This is for the password and it also has a function to check if the password matches and a function to encrypt the password ![](https://user-images.githubusercontent.com/69410126/232369674-a6719a7a-1b44-49e0-b3cb-afbf20f7f83f.png)
- These display the values of the Leaderboard in JSON values for analyzation, and CRUD functions later on
    - ![](https://user-images.githubusercontent.com/69410126/232369779-f10018e9-f7ce-43cc-a2f7-3c7ee761d9aa.png)
- These CRUD functions allow us to test out the Create, Read, Update, and Delete
    - ![](https://user-images.githubusercontent.com/69410126/232370060-6d36173d-414e-47f0-ace6-86880603dfc7.png)
    - This is the tester data that would populate the database in order to test the data ![](https://user-images.githubusercontent.com/69410126/232370169-b2632e88-459b-4a06-87b7-cf6b5e0d2c26.png)
    - The try and except allows the code to continue to function even if there is an error, if there is an error, it is reported but the init_leaderboards() still initializes all of the leaderboards after the error

In [ ]:
from flask import Flask
from flask_login import LoginManager
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from pathlib import Path
import os

"""
These object can be used throughout project.
1.) Objects from this file can be included in many blueprints
2.) Isolating these object definitions avoids duplication and circular dependencies
"""

# Setup of key Flask object (app)
app = Flask(__name__)
project_path = Path.cwd().as_posix()
# Setup SQLAlchemy object and properties for the database (db)
file_path = os.path.abspath(os.getcwd())+"/nighthawkguessr_api/volumes/sqlite.db"
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///'+file_path
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
app.config["SECRET_KEY"] = "SECRET_KEY"
db = SQLAlchemy(app)
Migrate(app, db)

# Images storage
app.config["MAX_CONTENT_LENGTH"] = 5 * 1024 * 1024  # maximum size of uploaded content
app.config["UPLOAD_EXTENSIONS"] = [".jpg", ".png", ".gif"]  # supported file types
app.config["UPLOAD_FOLDER"] = "volumes/uploads/"  # location of user uploaded content

from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.exc import IntegrityError
import json
from werkzeug.security import generate_password_hash, check_password_hash


class Leaderboard(db.Model):
    __tablename__ = "leaderboard"

    id = Column(Integer, primary_key=True)
    _username = Column(String(255), unique=True, nullable=False)
    _password = Column(String(255), nullable=False)
    _pointsEasy = Column(Integer, nullable=False)
    _pointsMedium = Column(Integer, nullable=False)
    _pointsHard = Column(Integer, nullable=False)

    def __init__(self, username, password, pointsEasy, pointsMedium, pointsHard):
        self._username = username
        self.set_password(password)
        self._pointsEasy = pointsEasy
        self._pointsMedium = pointsMedium
        self._pointsHard = pointsHard

    def __repr__(self):
        return "<Leaderboard(id='%s', username='%s', pointsEasy='%s', pointsMedium='%s', pointsHard='%s')>" % (
            self.id,
            self.username,
            self.pointsEasy,
            self.pointsMedium,
            self.pointsHard,
        )

    @property
    def username(self):
        return self._username

    @username.setter
    def username(self, value):
        self._username = value

    def is_username(self, username):
        return self._username == username

    @property
    def pointsEasy(self):
        return self._pointsEasy

    @pointsEasy.setter
    def pointsEasy(self, value):
        self._pointsEasy = value

    @property
    def pointsMedium(self):
        return self._pointsMedium

    @pointsMedium.setter
    def pointsMedium(self, value):
        self._pointsMedium = value

    @property
    def pointsHard(self):
        return self._pointsHard

    @pointsHard.setter
    def pointsHard(self, value):
        self._pointsHard = value

    @property
    def password(self):
        return self._password[0:10] + "..."

    def set_password(self, password):
        self._password = generate_password_hash(password, method='sha512')

    def is_password(self, password):
        result = check_password_hash(self._password, password)
        if result:
            return True
        else:
            return False

    def to_dict(self):
        return {"id": self.id, "username": self.username, "password": self.password, "pointsEasy": self._pointsEasy, "pointsMedium": self._pointsMedium, "pointsHard": self._pointsHard}

    def __str__(self):
        return json.dumps(self.read())

    def create(self):
        try:
            db.session.add(self)
            db.session.commit()
            return self
        except IntegrityError:
            db.session.remove()
            return None

    def read(self):
        return {
            "id": self.id,
            "username": self.name,
            "password": self.uid,
            "pointsEasy": self.pointsEasy,
            "pointsMedium": self.pointsMedium,
            "pointsHard": self.pointsHard
        }

    def update(self, username="", password="", pointsEasy="", pointsMedium="", pointsHard=""):
        """only updates values with length"""
        if len(username) > 0:
            self.username = username
        if len(pointsEasy) > 0:
            self.pointsEasy = pointsEasy
        if len(pointsMedium) > 0:
            self.pointsMedium = pointsMedium
        if len(pointsHard) > 0:
            self.pointsHard = pointsHard
        if len(password) > 0:
            self.set_password(password)
        db.session.add(self)
        db.session.commit()
        return self

    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None


def init_leaderboards():
    """Create database and tables"""
    db.create_all()
    """Tester data for table"""
    l1 = Leaderboard(username="bob", password="apple",
                     pointsEasy=2, pointsMedium=5, pointsHard=3)
    l2 = Leaderboard(username="bobby", password="appley",
                     pointsEasy=20, pointsMedium=50, pointsHard=30)
    l3 = Leaderboard(username="bobbert", password="appled",
                     pointsEasy=200, pointsMedium=500, pointsHard=300)
    l4 = Leaderboard(username="bobruth", password="appler",
                     pointsEasy=100, pointsMedium=300, pointsHard=500)
    leaderboards = [l1, l2, l3, l4]
    print(leaderboards)

    """Builds sample user/note(s) data"""
    for l in leaderboards:
        try:
            '''add user to table'''
            object = l.create()
            print(f"Created new uid {object.username}")
            db.session.add(l)
            db.session.commit()
        except:
            '''fails with bad or duplicate data'''
            print(f"Records exist uid {l.username}, or error.")

## JWT (Nikhil)

## Picture Database (Ethan T.)
- Used CRUD methods which have create, read, update, and delete rows in the table.  There is a function called initEasyImages which populates the 'Images' table with data.
- The __init__ method is used to create a new row in the table with a given imagePath, xCoord, yCoord, and difficulty. The __repr__ method returns a string representation of the object when it is printed.
-  The table has five columns: id, _imagePath, _xCoord, _yCoord, and _difficulty.



### Picture Encoding (Alex)

## Connection of Databases from Frontend to Backend (David)
We will establish endpoints in the backend, which will ultimately connect to the frontend

The backened takes note of the SQLite database that we use, and runs certain queries over it to filter and return valid values. Additionally, we also used a dictionary data structure to represent the key and value pairs in our JSON data, and also used a list to represent all the valid database entires that we can select from. By creating an API endpoint on our backend, we can thus retrieve this jsondata in the frontend, and display the necessary image or update the necessary variables that is required for program execution

### Methods in Backend

In [ ]:
from flask import Blueprint, request
from flask_restful import Api, Resource, reqparse

leaderboard_bp = Blueprint("leaderboards", __name__)
leaderboard_api = Api(leaderboard_bp)

def get_user_list():
    users_list = [[user._username, int(user._pointsEasy)+2*int(user._pointsMedium)+3*int(user._pointsHard)] for user in Leaderboard.query.all()]
    return users_list

def find_by_username(username):
    users = Leaderboard.query.filter_by(_username=username).all()
    return users[0]


class LeaderboardAPI(Resource):
    def get(self):
        username = request.get_json().get("username")
        print(username, "uid")
        user = find_by_username(username)
        if user:
            return user.to_dict()
        return {"message": user}, 404

    def post(self):
        parser = reqparse.RequestParser()
        parser.add_argument("username", required=True, type=str)
        parser.add_argument("password", required=True, type=str)
        parser.add_argument("pointsEasy", required=True, type=int)
        parser.add_argument("pointsMedium", required=True, type=int)
        parser.add_argument("pointsHard", required=True, type=int)
        args = parser.parse_args()

        leaderboard = Leaderboard(args["username"], args["password"],
                                  args["pointsEasy"], args["pointsMedium"], args["pointsHard"])
        try:
            db.session.add(leaderboard)
            db.session.commit()
            return leaderboard.to_dict(), 201
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500

    def put(self):
        username = request.get_json().get("username")
        print(username, "uid")

        try:
            user = find_by_username(username)
            if user:
                user.pointsEasy = int(request.get_json().get("pointsEasy"))
                user.pointsMedium = int(request.get_json().get("pointsMedium"))
                user.pointsHard = int(request.get_json().get("pointsHard"))
                db.session.commit()
                return user.to_dict(), 201
            else:
                return {"message": "leaderboard not found"}, 404
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500

    def delete(self):
        username = request.get_json().get("username")
        print(username, "uid")

        try:
            user = find_by_username(username)
            if user:
                db.session.delete(user)
                db.session.commit()
                return user.to_dict()
            else:
                return {"message": "leaderboard not found"}, 404
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500


class LeaderboardListAPI(Resource):
    def get(self):
        try:
            leaderboards = db.session.query(Leaderboard).all()
            return [leaderboard.to_dict() for leaderboard in leaderboards]
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500

    def delete(self):
        try:
            db.session.query(Leaderboard).delete()
            db.session.commit()
            return []
        except Exception as e:
            db.session.rollback()
            return {"message": f"server error: {e}"}, 500
        
class LeaderboardTop10(Resource):
    def partition(self, arr, lo, hi):
        pivot = arr[hi][1]
        i = lo - 1
        for j in range(lo, hi):
            if arr[j][1] >= pivot:
                i = i + 1
                (arr[i], arr[j]) = (arr[j], arr[i])
        (arr[i + 1], arr[hi]) = (arr[hi], arr[i + 1])
        return i+1
    
    def qSortUserList(self, arr, lo, hi):
        if lo < hi:
            part = self.partition(arr, lo, hi)
            self.qSortUserList(arr, lo, part-1)
            self.qSortUserList(arr, part+1, hi)

    def get(self):
        users_list = get_user_list()
        top10 = {}
        self.qSortUserList(users_list, 0, len(users_list)-1)
        for user in users_list:
            top10[user[0]] = user[1]
        print(top10)
        if len(top10) <= 10:
            return top10
        return top10[:10]

### Frontend and How it Communicates

In [ ]:
const isLocalhost = Boolean(
        window.location.hostname === "localhost" ||
        window.location.hostname === "[::1]" ||
        window.location.hostname.match(/^127(?:\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}$/)
      )
      const api = isLocalhost ? "http://localhost:8200" : "";

      function addData(username, password, pointsEasy, pointsMedium, pointsHard){

      let data = {
        "username": username,
        "password": password,
        "pointsEasy": pointsEasy,
        "pointsMedium": pointsMedium,
        "pointsHard": pointsHard
      }

      fetch(api + '/leaderboard', {
        method: 'POST',
        headers: {
          'Content-Type': 'application/json',
        },
        body: JSON.stringify(data),
      })
        .then((response) => response.json())
        .catch((error) => {
          console.error('Error:', error);
        });
      }

      const getList = async () => {
        const list = await fetch(api + "/leaderboardTop10").then((r) => r.json());
        let users = Object.entries(list);
        console.log(users)
        return list
      };

      getList().then(list => {
        list.forEach(cls => {
          addTask(cls.key, cls.value)
        })
      })

      function addTask(key, value) {
  
        var tableCells = [key, value]
        var row = document.createElement('tr')
        for (var i = 0; i < tableCells.length; i++) {
          var tableCell = document.createElement('th')
          tableCell.textContent = tableCells[i]
          tableCell.className = 'cell'
          if (i === 0) {
            tableCell.id = 'period'
          } else if (i === 1) {
            tableCell.id = 'class'
          } else if (i === 2) {
            tableCell.id = 'classNum'
          } else if (i === 3) {
            tableCell.id = 'classStart'
          } else if (i === 4) {
            tableCell.id = 'classEnd'
          }
          row.appendChild(tableCell)
        }
        schedule.appendChild(row)
      }

      getList()

### Leaderboard

>CRUD is an acronym used to describe the processes needed in a functional program. These include being able to create, read, update, and delete data based on the database used. This allows total control over data sets and allows fro total manipulation of the dataset.

#### Create

![]({{site.baseurl}}/lessonimages/create.png " ")

Parser is used to look through dataset and make sure there are no duplicates and then to make a new data entry.

#### Read

![]({{site.baseurl}}/lessonimages/read.png " ")

Parser is used in order to look through dataset for all values and send them to frontend.

#### Update

![]({{site.baseurl}}/lessonimages/update.png " ")

Parser is used to look for specific data value you are searching for and then update another specific value based on that found data fragment.

#### Delete

![]({{site.baseurl}}/lessonimages/delete.png " ")

Parser is used to look for specific data value you are searching for and then delete that found data fragment.

### How Images are Called and Displayed on Frontend

In [ ]:
<body style = "background-repeat: no-repeat; background-attachment: fixed; background-size: cover;">
</body>
<script>
const isLocalhost = Boolean(
  window.location.hostname === "localhost" ||
  window.location.hostname === "[::1]" ||
  window.location.hostname.match(/^127(?:\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}$/)
)
const api = isLocalhost ? "http://localhost:8200" : "";

const getList = async () => {
  const list = await fetch(api + "/api/images/GetEasyImage").then((r) => r.json());
  return list
};

getList().then(list => {
  document.getElementsByTagName("body")[0].style = "background-image: url('data:image/png;base64, " + list.bytes +"');"
})
</script>

The bytes of the images are called into the frontend with a fetch command, which pulls the data either from the locally or globally run flask server. The data is stored in ```list``` and is then made into a variable that is inserted directly into the HTML style of the body of the document, with the user of a css function that converts base64 into an image.

## Time and Space Complexity of Algorithms (Ethan Z, Alex)
The lesson for time and space complexity of algorithms will consist of demonstrations of sorting algorithms and the different time complexities that they come with on a small scale. This can be acomplished using things such as a deck of cards or even with actual people. An example of how this will work is laying out the cards in a random order on the table, and demonstrating different ways of sorting it. From methods such as bubble sort to methods such as bogo sort, it will be really easy to see exactly how much time it would take for these different sorting algoritms to complete. Then after that, it is easy to understand the concept of time complexity when given a real world example.

Space complexity can be demonstrated by using the same method, however, adding in the extra step of having the cards in a pile. This will show how much space is needed to complete the sorting algorithm. This will easily reveal the concept of space complexity because it allows people to witness a real world, physical example of the concept, turning a really hard to grasp concept into something that is easy to understand.

Additionally, we are also going to analyze certain algorithms within our project to show how such analysis is applicable to real world projects.